# Introduction to Automated Prompt Optimization

This notebook provides an introduction to using the adaptive learning features of our prompt optimization framework. We'll explore:

1. Setting up a PromptLibrary with sample templates
2. Creating a PerformanceAnalyzer to track template usage
3. Using the AdaptiveLearningManager to optimize prompts
4. Visualizing the optimization process and results

In [ ]:
# Import core components
import os
import json
import time
import asyncio
import matplotlib.pyplot as plt
import pandas as pd
from pprint import pprint

# Import our framework components
from ailf.cognition.prompt_library import PromptLibrary
from ailf.schemas.prompt_engineering import PromptTemplateV1, PromptLibraryConfig
from ailf.feedback.performance_analyzer import PerformanceAnalyzer
from ailf.feedback.adaptive_learning_manager import AdaptiveLearningManager

## Setup: Creating a Sample Prompt Library

First, let's create a temporary directory and set up a PromptLibrary with some sample templates.

In [ ]:
# Create a temporary directory for our prompt templates
import tempfile
prompt_library_path = tempfile.mkdtemp()
print(f"Created temporary directory for templates: {prompt_library_path}")

# Initialize configuration for the prompt library
config = PromptLibraryConfig(
    library_path=prompt_library_path,
    default_prompt_id="weather_query",
    auto_save=True
)

In [ ]:
# Define sample templates
templates = [
    {
        "template_id": "weather_query",
        "version": 1,
        "description": "A template for querying weather information",
        "system_prompt": "You are a helpful weather assistant.",
        "user_prompt_template": "What's the weather like in {location} today?",
        "placeholders": ["location"],
        "tags": ["weather", "query"],
        "created_at": time.time()
    },
    {
        "template_id": "news_summary",
        "version": 1,
        "description": "A template for summarizing news articles",
        "system_prompt": "You are a helpful news summarizer.",
        "user_prompt_template": "Summarize this news article: {article}",
        "placeholders": ["article"],
        "tags": ["news", "summary"],
        "created_at": time.time()
    },
    {
        "template_id": "product_recommendation",
        "version": 1,
        "description": "A template for recommending products",
        "system_prompt": "You are a product recommendation assistant.",
        "user_prompt_template": "Recommend products for {category} that match {preferences}",
        "placeholders": ["category", "preferences"],
        "tags": ["products", "recommendations"],
        "created_at": time.time()
    }
]

# Save templates to JSON files
for template_data in templates:
    filename = f"{template_data['template_id']}_v{template_data['version']}.json"
    filepath = os.path.join(prompt_library_path, filename)
    with open(filepath, 'w') as f:
        json.dump(template_data, f, indent=2)
    print(f"Created template file: {filename}")

In [ ]:
# Initialize the PromptLibrary
prompt_library = PromptLibrary(config)

# List available templates
print("Available templates:")
for template_id in prompt_library.list_template_ids():
    template = prompt_library.get_template(template_id)
    print(f"- {template.template_id} (v{template.version}): {template.description}")

## Creating a Mock Performance Analyzer

Now, let's create a mock PerformanceAnalyzer that provides simulated performance data for our templates.

In [ ]:
# Define a simple mock PerformanceAnalyzer
class MockPerformanceAnalyzer(PerformanceAnalyzer):
    """A mock performance analyzer for demonstration purposes."""
    
    def __init__(self, mock_data=None):
        """Initialize with optional mock data."""
        self.mock_data = mock_data or {}
        
    def analyze_prompt_success(self):
        """Return mock prompt analysis data."""
        return self.mock_data.get("prompt_analysis", {})
        
    def get_general_metrics(self):
        """Return mock general metrics."""
        return self.mock_data.get("general_metrics", {})
        
    def find_prompt_correlations(self):
        """Return mock correlations."""
        return self.mock_data.get("correlations", {})

# Create mock performance data
mock_data = {
    "prompt_analysis": {
        "weather_query": {
            "total_uses": 100,
            "successful_outcomes": 60,
            "error_count": 40,
            "average_feedback_score": 0.3,
            "error_rate": 0.4,  # High error rate (>30%)
            "success_rate": 0.6
        },
        "news_summary": {
            "total_uses": 50,
            "successful_outcomes": 45,
            "error_count": 5,
            "average_feedback_score": 0.8,
            "error_rate": 0.1,
            "success_rate": 0.9
        },
        "product_recommendation": {
            "total_uses": 75,
            "successful_outcomes": 30,
            "error_count": 45,
            "average_feedback_score": 0.1,  # Low feedback score
            "error_rate": 0.6,
            "success_rate": 0.4
        }
    },
    "general_metrics": {
        "total_interactions": 225,
        "successful_interactions": 135,
        "failed_interactions": 90,
        "average_response_time": 2.3
    },
    "correlations": {
        "prompt_success_vs_length": 0.7
    }
}

# Create the performance analyzer
performance_analyzer = MockPerformanceAnalyzer(mock_data)

## Setting Up the AdaptiveLearningManager

Now let's set up the AdaptiveLearningManager to use our mock performance analyzer and prompt library.

In [ ]:
# Define configuration for the AdaptiveLearningManager
learning_config = {
    "error_rate_threshold": 0.3,       # Consider error rates above 30% as high
    "feedback_optimization_threshold": 0.5,  # Consider feedback scores below 0.5 as low
    "auto_optimize_prompts": True,     # Enable automatic optimization
    "use_ai_for_improvements": False,  # Use rule-based improvements (no AI engine here)
    "feedback_suggestion_threshold": 0.3  # Threshold for suggesting improvements based on feedback
}

# Initialize the AdaptiveLearningManager
learning_manager = AdaptiveLearningManager(
    performance_analyzer=performance_analyzer,
    prompt_library=prompt_library,
    config=learning_config
)

print("AdaptiveLearningManager initialized with configuration:")
print(f"- Error rate threshold: {learning_config['error_rate_threshold']}")
print(f"- Feedback optimization threshold: {learning_config['feedback_optimization_threshold']}")
print(f"- Auto-optimize prompts: {learning_config['auto_optimize_prompts']}")

## Analyzing Current Performance

Let's examine the performance data for our prompt templates.

In [ ]:
# Get performance analysis data
prompt_analysis = performance_analyzer.analyze_prompt_success()

# Convert to DataFrame for easier visualization
analysis_df = pd.DataFrame.from_dict(
    {k: {kk: vv for kk, vv in v.items() if kk in ['error_rate', 'success_rate', 'average_feedback_score']} 
     for k, v in prompt_analysis.items()},
    orient='index'
)

# Display the performance metrics
print("Current Performance Metrics:")
display(analysis_df)

# Create a visualization
fig, ax = plt.subplots(figsize=(10, 6))
analysis_df.plot(kind='bar', ax=ax)
ax.set_title('Prompt Template Performance Metrics')
ax.set_ylabel('Score')
ax.set_ylim(0, 1)
ax.axhline(y=learning_config['error_rate_threshold'], color='r', linestyle='--', label='Error Threshold')
ax.axhline(y=learning_config['feedback_optimization_threshold'], color='g', linestyle='--', label='Feedback Threshold')
ax.legend()
plt.tight_layout()
plt.show()

## Running the Adaptation Cycle

Now, let's run the adaptation cycle to optimize our prompt templates based on the performance data.

In [ ]:
# Define a function to run the adaptation cycle asynchronously
async def run_adaptation_cycle():
    # Get the initial state of templates
    print("Initial templates:")
    initial_templates = {}
    for template_id in prompt_library.list_template_ids():
        template = prompt_library.get_template(template_id)
        initial_templates[template_id] = template
        print(f"- {template_id} (v{template.version}): {template.user_prompt_template}")
    
    print("\nRunning optimization cycle...")
    # Run the adaptive learning cycle
    cycle_results = await learning_manager.run_learning_cycle(auto_optimize=True)
    
    # Print optimization results
    print("\nOptimization cycle completed:")
    print(f"- Timestamp: {time.ctime(cycle_results.get('timestamp'))}")
    print(f"- Cycle ID: {cycle_results.get('cycle_id')}")
    
    # Check for optimized prompts
    optimized_prompts = cycle_results.get('optimized_prompts', [])
    print(f"\nOptimized {len(optimized_prompts)} templates")
    
    # Display the updated templates
    print("\nUpdated templates:")
    for template_id in prompt_library.list_template_ids():
        template = prompt_library.get_template(template_id)
        initial = initial_templates.get(template_id)
        if template.version > initial.version:
            print(f"- {template_id} (v{template.version}, was v{initial.version}):")
            print(f"  Before: {initial.user_prompt_template}")
            print(f"  After:  {template.user_prompt_template}")
            if template.version_notes:
                print(f"  Notes:  {template.version_notes}")
            print()
        else:
            print(f"- {template_id} (unchanged at v{template.version})")
    
    return cycle_results

# Run the adaptation cycle and get results
cycle_results = await run_adaptation_cycle()

## Examining Optimization History

Let's check the optimization history to see what changes were made.

In [ ]:
# Get the optimization history
optimization_history = learning_manager.get_optimization_history()

# Display the optimization history
print(f"Found {len(optimization_history)} optimization records:")
for i, record in enumerate(optimization_history):
    print(f"\nOptimization {i+1}:")
    print(f"- Template: {record.get('template_id')}")
    print(f"- Version: {record.get('original_version')} → {record.get('new_version')}")
    print(f"- Timestamp: {time.ctime(record.get('timestamp'))}")
    print(f"- Changes: {record.get('changes', 'No change notes')}")
    
    # If metrics are available, show them
    metrics = record.get('metrics', {})
    if metrics:
        print("- Metrics that triggered optimization:")
        for k, v in metrics.items():
            if isinstance(v, (int, float)):
                print(f"  - {k}: {v:.2f}")
            else:
                print(f"  - {k}: {v}")

## Viewing Template Version History

Let's examine the version history of one of the optimized templates.

In [ ]:
# Define a function to display template version history
def display_template_history(template_id):
    history = prompt_library.get_version_history(template_id)
    
    print(f"Version history for template '{template_id}':")
    for version in history:
        print(f"\nVersion {version.version}:")
        print(f"- User prompt: {version.user_prompt_template}")
        print(f"- System prompt: {version.system_prompt}")
        
        # Display optimization metadata if available
        if hasattr(version, 'updated_by_component') and version.updated_by_component:
            print(f"- Updated by: {version.updated_by_component}")
        if hasattr(version, 'updated_at') and version.updated_at:
            print(f"- Updated at: {time.ctime(version.updated_at)}")
        if hasattr(version, 'optimization_source') and version.optimization_source:
            print(f"- Optimization source: {version.optimization_source}")
        if hasattr(version, 'version_notes') and version.version_notes:
            print(f"- Version notes: {version.version_notes}")

# Check the history of the weather_query template
display_template_history("weather_query")

## Running Another Optimization Cycle

Let's run another optimization cycle with updated performance data to see continuous optimization in action.

In [ ]:
# Update the mock performance data to simulate changes after first optimization
performance_analyzer.mock_data["prompt_analysis"]["weather_query"]["error_rate"] = 0.35  # Still high error rate
performance_analyzer.mock_data["prompt_analysis"]["product_recommendation"]["error_rate"] = 0.25  # Improved but still needs work
performance_analyzer.mock_data["prompt_analysis"]["news_summary"]["average_feedback_score"] = 0.45  # Now below threshold

# Run another adaptation cycle
print("Running second optimization cycle with updated performance data...\n")
second_cycle_results = await run_adaptation_cycle()

# Display optimization history after second cycle
print("\nUpdated optimization history:")
optimization_history = learning_manager.get_optimization_history()
print(f"Total optimizations performed: {len(optimization_history)}")

# Display a summary of all template versions
print("\nFinal template versions:")
for template_id in prompt_library.list_template_ids():
    history = prompt_library.get_version_history(template_id)
    latest = prompt_library.get_template(template_id)
    print(f"- {template_id}: {len(history)} versions, latest is v{latest.version}")

## Summary

In this notebook, we've demonstrated:

1. Setting up a PromptLibrary with sample templates
2. Creating a mock PerformanceAnalyzer with simulated performance data
3. Using the AdaptiveLearningManager to automatically optimize underperforming templates
4. Running multiple optimization cycles to see continuous improvement
5. Viewing the history and evolution of templates over time

This demonstrates the end-to-end workflow for automated prompt optimization within the framework.